# Policy Gradient的推导

- <font color=brown>符号说明：文中 $a_t和u_t$ 混用，都表示t时刻的action。</font>

## I. 原目标：最大化状态的效用期望值

### I.1 两种常见的目标展开方法

1. trajectory的期望rewards：
$$U(\theta)  = E_{\tau }R(\tau|\pi_{\theta}) = \sum_{\tau }P(\tau ;\theta )R(\tau )$$

2. 不同状态价值的均值：
$$\begin{align}
严格说应该是初始状态的价值期望值：\\
U(\theta) & = E_{s\sim d(s_0)}[V^{\pi_{\theta }}(s)]=\sum_{s_0\in \mathcal{S} }d(s_0)V^{\pi_{\theta }}(s_0)\\
也可以取s处于stationary\ distribution时的分布：\\
U(\theta) & = E_{s\sim d(s)}[V^{\pi_{\theta }}(s)]=\sum_{s\in \mathcal{S} }d(s)V^{\pi_{\theta }}(s)\\
\end{align}$$

### I.2 目标展开式暗含的假设条件

- 期望值合理的前提是随机变量的分布稳定。也就是说，trajectory展开式中，假设了trajectory的分布稳定；状态价值展开式中，假设了状态s的分布稳定。<font color=red>但这两个条件在现实条件下都不成立。</font>
- 在状态价值展开式中，理论上，如果agent与环境交互的次数极大的条件下，$d(s)$可以用策略对应的stationary distribution。但是这个条件在真实环境中通常也很难满足。
- 尽管他们的分布处于不稳定状态，实践中还是会用monte carol抽样法直接估计期望的方式来得到估计量。

## II. 直接用目标推导梯度

### II.1 用trajectory的期望rewards推导

#### II.1.1 梯度的基本公式

$$\begin{align}
\nabla _{\theta }U(\theta ) 
& = \nabla _{\theta } \sum_{\tau }P(\tau ;\theta )R(\tau ) \\
& =  \sum_{\tau }\nabla _{\theta }P(\tau ;\theta )R(\tau ), {\color{green}{这步是因为\theta带给\tau的随机性都体现在P(\tau;\theta)里面}} \\
& =  \sum_{\tau }P(\tau ;\theta )\frac{\nabla _{\theta }P(\tau ;\theta )}{P(\tau ;\theta )}R(\tau ) \\
& =  E_{\tau }\nabla _{\theta }logP(\tau ;\theta )R(\tau ) \\
\\
\nabla _{\theta }logP(\tau ;\theta ) 
& = \nabla _{\theta }log\left[ d(s_0)\prod_{t=0}^{H-1} P(s_{t+1}|s_t, u_t)*\pi_{\theta }(u_t|s_t) \right ]\\
& = \nabla _{\theta }\left[logd(s_0) + \sum_{t=0}^{H-1}logP(s_{t+1}|s_t, u_t) + \sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)\right ]\\
& = \underset{0}{ \underbrace{\nabla _{\theta }logd(s_0)}}  + \underset{0}{\underbrace{\nabla _{\theta }\sum_{t=0}^{H-1}logP(s_{t+1}|s_t, u_t)}} + \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)\\
& = \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t) \\
\end{align}$$

$$\begin{align}
取梯度：g & = \nabla _{\theta }U(\theta )\\
& =  E_{\tau }\nabla _{\theta }logP(\tau ;\theta )R(\tau ) \\
& = E_{\tau }\left[\nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)R(\tau )\right ]\\
& = E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\sum_{t=0}^{H-1}r(s_t, u_t)\right ]
\end{align}$$

#### II.1.2 rewards to go

$$g = E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\sum_{t'=t}^{H-1}r(s_{t'}, u_{t'})\right ]$$

#### II.1.3 baseline

- 当b是常数时，下式成立：
$$\begin{align}
E_{\tau }[\nabla _{\theta }logP(\tau ;\theta )]b& = \nabla _{\theta } \sum_{\tau }P(\tau ;\theta )b \\
& = b\nabla _{\theta } \sum_{\tau }P(\tau ;\theta )\\
& = b\nabla _{\theta }1 = 0
\end{align}$$

- <font color=norange>**即使b是states的函数，只要b(s)不受action的影响，等式仍然成立。注意，这里关键是b(s)与action的关系，而不是b(s)与策略的关系。即使b(s)受策略的影响，比如图取值为状态价值函数V(s)也不改变结果。**</font>见下面分析过程：

$$\begin{align}
E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)b(s_t)\right ]
& = E_{\tau }\left[ \sum_{t=0}^{H-1}[\nabla _{\theta }log\pi_{\theta }(a_t|s_t)]V^{\pi }(s_t)\right ]\\
& = \underset{s_t\in\mathcal{S},a_t\in \mathcal{A}}{E}\left[ \sum_{t=0}^{H-1}[\nabla _{\theta }log\pi_{\theta }(a_t|s_t)]V^{\pi }(s_t)\right ]\\
& = \sum_{t=0}^{H-1}\left[\sum_{s_t\in\mathcal{S}}P(s_t)\sum_{a_t\in \mathcal{A}} [\pi_{\theta }(a_t|s_t)\nabla _{\theta }log\pi_{\theta }(a_t|s_t)]V^{\pi }(s_t)\right ]\\
& = \sum_{t=0}^{H-1}\left[\sum_{s_t\in\mathcal{S}}P(s_t)\sum_{a_t\in \mathcal{A}}[\nabla _{\theta }\pi_{\theta }(a_t|s_t)]V^{\pi }(s_t)\right ],{\color{green}{因为V(s_t)与a_t无关}}\\
& = \sum_{t=0}^{H-1}\left[\sum_{s_t\in\mathcal{S}}P(s_t)V^{\pi }(s_t)\sum_{a_t\in \mathcal{A}}\nabla _{\theta }\pi_{\theta }(a_t|s_t)\right ]\\
& = \sum_{t=0}^{H-1}\left[\sum_{s_t\in\mathcal{S}}P(s_t)V^{\pi }(s_t)\nabla _{\theta }\left(\sum_{a_t\in \mathcal{A}}\pi_{\theta }(a_t|s_t)\right)\right ]\\
& = \sum_{t=0}^{H-1}\left[\sum_{s_t\in\mathcal{S}}P(s_t)V^{\pi }(s_t)\nabla _{\theta }1\right ]=0
\end{align}$$

- 为了降低估计量的方差，b(s)的sub-optimal取值为$b(s)=V^{\pi}(s)$。
  - 将action advantage记为$A_t=Q^{\pi}(s_t,u_t)-V^{\pi}(s_t)$
  - 因为 $Q^{\pi}(s_t,u_t) = \sum_{t'=t}^{H-1}E_{\pi}[r(s_{t'}, u_{t'})|s_t,u_t]$
$$\begin{align}
g & =  E_{\tau }\nabla _{\theta }logP(\tau ;\theta )[R(\tau ) - b]\\
&= E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\left( \sum_{t'=t}^{H-1}r(s_{t'}, u_{t'})-b(s_t)\right)\right ]\\
&= E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\left( Q(s_{t}, u_{t})-V(s_t)\right)\right ]\\
&= E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)A_t\right ]\\
\end{align}$$

- 此时，策略梯度的估计量为：
$$\begin{align}
\hat g & = \frac{1}{N} \sum _{i=1 }^N\nabla _{\theta }logP(\tau_i;\theta )[R(\tau_i) - b]\\
&= \frac{1}{N} \sum _{i=1 }^N\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\left( \sum_{t'=t}^{H-1}r(s_{t'}^{(i)}, u_{t'}^{(i)})-V^{\pi}(s_t^{(i)})\right)\right ]\\
&= \frac{1}{N} \sum _{i=1 }^N\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\hat A_t\right ]
\end{align}$$

- <font color=blue>**不同估计方法的bias-varince tradeoff**</font>：
  1. 如果用Monte Carol估$R(\tau_i) - V(s_t^{(i)})$，得到的估计量unbiased，但是varince很大
  2. 改用DNN先估 $V(s_t)或者Q(s_t)$，然后得到Advantage function的估计量的方法能降低variance，但是此时不再满足unbiased条件。

### II.2 用state function期望值推导

- <font color=orange>这种推导方式的假设比用trajectory的假设多了一条：d(s)不受策略影响。而这一点在现实中通常不成立，所以这种方式描述现实的能力不如用trajectory期望值的推导方式。</font>

- <font color=red>**假如d(s)不受策略影响**</font>：这个假设通常不成立，但是能简化策略梯度的推导
$$\begin{align}
g = \nabla _{\theta }U(\theta) & = \nabla _{\theta }E_{s\sim d(s)}[V^{\pi_{\theta }}(s)] \\
& = \nabla _{\theta }\sum_{s\in \mathcal{S} }d(s)V^{\pi_{\theta }}(s) \\
& = \sum_{s\in \mathcal{S} }d(s)\nabla _{\theta }V^{\pi_{\theta }}(s) \\
\end{align}$$

- 将状态价值函数的梯度展开可以得到一个线性方程组：
$$\begin{align}
\nabla _{\theta }V^{\pi_{\theta }}(s)
& = \nabla _{\theta }\left[ \sum_{a\in \mathcal{A}} \pi(a|s)Q^{\pi }(s,a)\right]\\
& = \sum_{a\in \mathcal{A}} \left[ Q^{\pi }(s,a)\nabla _{\theta } \pi(a|s) + \pi(a|s)\nabla _{\theta } Q^{\pi }(s,a)\right]\\
& = \sum_{a\in \mathcal{A}} \left[ Q^{\pi }(s,a)\nabla _{\theta } \pi(a|s) + \pi(a|s)\nabla _{\theta } \left (r(s,a,s') + \gamma \sum_{s'\in \mathcal{S}}P(s'|s,a) V^{\pi }(s')\right )\right]\\
& = \sum_{a\in \mathcal{A}} \left[ Q^{\pi }(s,a)\nabla _{\theta } \pi(a|s) + \pi(a|s) \gamma \sum_{s'\in \mathcal{S}}P(s'|s,a)\nabla _{\theta }V^{\pi }(s')\right]\\
& = \sum_{a\in \mathcal{A}}  Q^{\pi }(s,a)\nabla _{\theta } \pi(a|s) + \gamma \sum_{a\in \mathcal{A}}\pi(a|s) \sum_{s'\in \mathcal{S}}P(s'|s,a)\nabla _{\theta }V^{\pi }(s')\\
& = \sum_{a\in \mathcal{A}}  Q^{\pi }(s,a)\nabla _{\theta } \pi(a|s) + \gamma \sum_{s'\in \mathcal{S}}P^{\pi}(s'|s)\nabla _{\theta }V^{\pi }(s')\\
\end{align}$$

- 求解该线性方程组的解为：<font color=brown>[详见math of RL lemma 9.2]</font>
$$\begin{align}
\nabla _{\theta }V^{\pi_{\theta }}(s)
& = \sum_{s'\in \mathcal{S}}Pr^{\pi}(s'|s)\sum_{a\in\mathcal{A}}\nabla_{\theta}\pi(a|s')Q(s',a)\\
其中：\\
Pr^{\pi}(s'|s) & =\sum_{k=0}^{\infty}\gamma^k[P^{\pi,k }(s'|s)],\ P^{\pi,k }(s'|s)是s经k步转向s'的概率
\end{align}$$

- 代入策略梯度式：
$$\begin{align}
g & = \sum_{s\in \mathcal{S} }d(s)\nabla _{\theta }V^{\pi_{\theta }}(s) \\
& = \sum_{s\in \mathcal{S} }d(s)\sum_{s'\in \mathcal{S}}Pr^{\pi}(s'|s)\sum_{a\in\mathcal{A}}\nabla_{\theta}\pi(a|s')Q(s',a)\\
& = \sum_{s'\in \mathcal{S}}\left(\sum_{s\in \mathcal{S} }d(s)Pr^{\pi}(s'|s)\right)\sum_{a\in\mathcal{A}}\nabla_{\theta}\pi(a|s')Q(s',a)\\
& = \sum_{s'\in \mathcal{S}}\rho (s')\sum_{a\in\mathcal{A}}\nabla_{\theta}\pi(a|s')Q(s',a)\ ,\ 更换符号\\
& = \sum_{s\in \mathcal{S}}\rho (s)\sum_{a\in\mathcal{A}}\nabla_{\theta}\pi(a|s)Q(s,a) \\
& = \sum_{s\in \mathcal{S}}\rho (s)\sum_{a\in\mathcal{A}}\pi_{\theta}(a|s)\nabla_{\theta}log\pi_{\theta}(a|s)Q^{\pi}(s,a) \\
& = \underset{s\sim \rho^{\pi}, a\sim \pi}{E}\nabla_{\theta}log\pi_{\theta }(a|s)Q^{\pi }(s,a) \\
\end{align}$$

## III. Importance sampling形式的目标和梯度

### III.1 基于trajectory的期望rewards的Importance sampling

#### III.1.1 目标$J(\theta)$的Importance sampling

$$\begin{align}
J (\theta )& = \underset{\tau\sim \pi _\theta}{E} [R(\tau)] = \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  [\frac{P_{\theta}(\tau)}{P_{\bar \theta}(\tau)}R(\tau)] \\
其中：\\
\frac{P_{\theta}(\tau)}{P_{\bar \theta}(\tau)} & = \frac{P(s_0)\prod_{t=0}^{H-1}\pi_{\theta }(a_t|s_t)P(s_{t+1}|s_t, a_t)}{P(s_0)\prod_{t=0}^{H-1}\pi_{\bar \theta }(a_t|s_t)P(s_{t+1}|s_t, a_t)} 
= \frac{\prod_{t=0}^{H-1}\pi_{\theta }(a_t|s_t)}{\prod_{t=0}^{H-1}\pi_{\bar \theta }(a_t|s_t)} 
= \prod_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar \theta }(a_t|s_t)}  \\
代入上式：\\
J(\theta) &= \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  [\frac{P_{\theta}(\tau)}{P_{\bar \theta}(\tau)}R(\tau)]= \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  [ \prod_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar \theta }(a_t|s_t)} R(\tau)]
\end{align}$$

#### III.1.2 梯度$\nabla_\theta J(\theta)$的Importance sampling

$$\begin{align}
\nabla _{\theta }J(\theta) & =  \underset{\tau\sim \pi_{\theta}}{E} \ \  \nabla _{\theta }logP(\tau ;\theta )R(\tau ) \\
& = \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  \frac{P_{\theta}(\tau)}{P_{\bar \theta}(\tau)}\nabla _{\theta }logP_{\theta}(\tau)R(\tau )  \\
& = \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  \frac{P_{\theta}(\tau)}{P_{\bar \theta}(\tau)}\nabla _{\theta }log\pi _{\theta}(\tau)R(\tau )  \\
&= \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  \left [\left( \prod_{t=0}^{H-1}\frac{\pi_{\theta,t }}{\pi_{\bar \theta, t }} \right)\left(\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta,t }\right)\left(\sum_{t=0}^{H-1}r_t\right)\right]\\
&= \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  \left [\left( \prod_{t^{''}=0}^{t}\frac{\pi_{\theta,t^{''} }}{\pi_{\bar \theta,t^{''} }} \right)\left( \prod_{t^{'''}={t+1
}}^{H-1}\frac{\pi_{\theta, t^{'''} }}{\pi_{\bar \theta, t^{'''} }} \right)\left(\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta,t }\sum_{t'=t}^{H-1}r_t\right)\right]\\
&= \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  \left [\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta,t }(\prod_{t^{''}=0}^{t}\frac{\pi_{\theta ,t^{''} }}{\pi_{\bar \theta ,t^{''}}})(\sum_{t'=t}^{H-1}r_t)\right]
\end{align}$$

- 实践中不用这个式子，因为上式中：
$$\prod_{t^{''}=0}^{t}\frac{\pi_{\theta ,t^{''} }}{\pi_{\bar \theta ,t^{''}}} \rightarrow exponential\ in\  T，方差极大$$

### III.2 IS梯度的近似和它的surrogate loss目标

#### III.2.1 用state-action的的边际分布得到Importance sampling梯度的近似

- 另一种推导方式：假设$(s_t, a_t)\sim P_{\theta}(s_t, a_t)$是state-action的marginal distribution。
- 这种形式的优点是没有累乘，所以没有前一种推导方式中方差太大的问题。代价是得到的是策略梯度的近似估计量。实践中可用的原因是，尽管是近似估计量，但它导致的估计误差有明确的上限。

- 推导式：
$$\begin{align}
g & = \underset{\tau \sim \pi_{\theta }}{E} \left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})\right ]\\
 & = \underset{\tau \sim \pi_{\theta }}{E} \left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t)\right ]\\
& = \sum_{t=0}^{H-1}\underset {(a_t, s_t)\sim P_{\theta }(a_t, s_t)}{E} \nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t)\\
& = \sum_{t=0}^{H-1}\sum_{a_t\in\mathcal{A},s_t\in\mathcal{S}} P_{\theta }(a_t,s_t)\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t)\\
& = \sum_{t=0}^{H-1}\sum_{a_t\in\mathcal{A},s_t\in\mathcal{S}} P_{\bar \theta }(a_t,s_t)\frac{P_{\theta }(a_t,s_t)}{P_{\bar \theta }(a_t,s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t)\\
& = \sum_{t=0}^{H-1}\underset {(a_t, s_t)\sim P_{\bar \theta }(a_t, s_t)}{E}\frac{P_{\theta }(a_t,s_t)}{P_{\bar \theta }(a_t,s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t) \\
& = \sum_{t=0}^{H-1}\underset {(a_t, s_t)\sim P_{\bar \theta }(a_t, s_t)}{E}\frac{P_{\theta }(s_t)\pi_{\theta }(a_t|s_t)}{P_{\bar \theta }(s_t)\pi_{\bar\theta }(a_t|s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t) \\
& \approx \sum_{t=0}^{H-1}\underset {(a_t, s_t)\sim P_{\bar \theta }(a_t, s_t)}{E}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t) \\
& = \underset {\tau \sim \pi_{\bar\theta }}{E}\sum_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})
\end{align}$$

- 上面推导过程的说明：
  - 第二个等式成立是因为：将$\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})$替换为$Q^{\pi_{\theta }}(s_t,a_t)$在trajectory的期望条件下是成立的。
  - 第三个等式成立是因为：$\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t)$ 是$(s_t,a_t)$为变量的函数。
    1. $\pi_{\theta }(a_t|s_t)$ 是$(s_t,a_t)$为变量的函数
    2. $Q^{\pi_{\theta }}(s_t,a_t)$的展开式中虽然有$t'>t$的所有time-steps的影响，但是所有的$a_{t'},s_{t'}$的不确定性都被期望积掉了。
  - 约等式中直接去掉了$\frac{P_{\theta}(s_t)}{P_{\bar \theta}(s_t)}$，这是导致最终近似估计式简化的原因，也是导致估计量近似而不是等式的原因。
  - <font color=blue>**最后一个等式中，均值条件从 $(a_t, s_t)\sim P_{\bar \theta }(a_t, s_t)换成\tau\sim\pi_{\bar\theta}$ 之后，action function也恢复成rewards累加形式。这里的细微差异是，当取action function形式时，它是策略$\pi_{\theta}$ 条件下的action function；但恢复成rewards形态后，就是直接用每个time-steps上的reward加和来计算。后者只是trajectory上的state-action pair的函数，不受策略影响，因此，做importance sampling抽样估计时就可以用策略$\pi_{\bar \theta}$抽样的样本来计算了。要注意这一点细微的差异。**</font>

#### III.2.2 surrogate loss目标

- 取梯度为IS梯度近似表达式时，为了利用现成的深度学习框架，可以用它反推一个surrogate loss

$$\begin{align}
当梯度取下式时：\\
g & = \underset {\tau \sim \pi_{\bar\theta }}{E}\sum_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})\\
对应的目标可以用：\\
L^{IS}(\theta ) & = \underset {\tau \sim \pi_{\bar\theta }}{E}\sum_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})\\
对L(\theta)求梯度很容易得到：\\
& \nabla_{\theta }L^{IS}(\theta ) = g
\end{align}$$

- 用梯度反推得到的surrogate loss可以从importance sampling角度来理解其含义：
  - 严格地说，这里的surrogate loss不算真正的importance sampling loss，因为trajectory的rewards如果要取importance sampling的期望值的话，应该用$(s_t, a_t)$的边际分布来计算importance weight，但这里用的是$\pi(a_t|s_t)$。
  - 但一般会将这个loss用importance sampling来标记，这是因为它是真正的importance sampling loss的近似值，实践中PG使用的loss也是它。<font color=brown>[详见后文：'用Policy Gradient近似policy iteration的误差分析'部分]</font>
$$\begin{align}
L^{IS}(\theta ) & = \underset {\tau \sim \pi_{\bar\theta }}{E}\sum_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})\\
& = \sum_{t=0}^{H-1}E_{s_t\sim P_{\bar \theta }(s_t)}E_{a_t\sim {\pi_{\bar \theta }}}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})\\
\end{align}$$

## IV. 从policy iteration角度理解policy gradient(PG)：为什么PG能improve policy?

- <font color=blue>**思路**：</font> <font color=green>先证明policy iteration是在优化正确的RL目标。然后证明policy gradient的目标和policy iteration的目标非常接近，在一定条件下这两个目标的差异可以控制在一个明确的bound之内。既然policy iteration能让策略在迭代过程中improvement，那么与之近似的policy gradient也应该在优化正确的目标。</font>

#### III.4.1 优化目标的重新表述

- **claim**: 
  - 符号：$\bar\theta$表示new policy参数
$$J(\bar \theta)-J(\theta)=\underset{\tau\sim P_{\bar \theta}(\tau)}{E}\left[\sum_t\gamma^tA^{\pi_{\theta}}(s_t,a_t) \right]$$

- 证明：
$$\begin{align}
J(\bar \theta)-J(\theta) & = J(\bar \theta)- E_{s_0\sim P(s_0)}[V^{\pi_{\theta }}(s_0)]，这里P(s_0)不取决于\theta \\
& = J(\bar \theta)- E_{\tau\sim P_{\bar\theta}(\tau)}[V^{\pi_{\theta }}(s_0)]，\tau的分布不改变s_0的边际分布，所以\tau可以改取任意分布，只要它的初始状态满足P(s_0)就行\\
& = J(\bar \theta)- E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty}\gamma^tV^{\pi_{\theta }}(s_t)-\sum_{t=1}^{\infty}\gamma^tV^{\pi_{\theta }}(s_t)\right]\\
& = J(\bar \theta)- E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty}\left(\gamma^tV^{\pi_{\theta }}(s_t)-\gamma^{t+1}V^{\pi_{\theta }}(s_{t+1})\right)\right]\\
& = J(\bar \theta)+ E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty}\gamma^t\left(\gamma V^{\pi_{\theta }}(s_{t+1})-V^{\pi_{\theta }}(s_t)\right)\right]\\
&= E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^tr(s_t,a_t)\right]+ E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty}\gamma^t\left(\gamma V^{\pi_{\theta }}(s_{t+1})-V^{\pi_{\theta }}(s_t)\right)\right]\\
&= E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^t\left(r(s_t,a_t)+\gamma V^{\pi_{\theta }}(s_{t+1})-V^{\pi_{\theta }}(s_t)\right)\right]\\
& = E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^tA^{\pi_{\theta}}(s_t,a_t) \right]
\end{align}$$

- <font color=red>说明：上面第二个等式成立是因为$P(s_0)$是环境决定的，不受策略影响。因此期望值可以取任意分布，只要满足该分布下$s_0$的边际分布是$P(s_0)$就行。</font>

- 该等式的意义：说明policy iteration带来的策略更新$\bar \theta$能最大化RL的优化目标$J(\bar \theta)$。
  - 用数学表述为：
$$\underset{\bar \theta }{max}\ E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^tA^{\pi_{\theta}}(s_t,a_t) \right]
\overset{等价于}{\Longrightarrow} \underset{\bar \theta }{max}J(\bar \theta)-J(\theta)\overset{等价于}{\Longrightarrow} \underset{\bar \theta }{max}J(\bar \theta)$$
  - 从而证明了，policy iteration做了正确的事情。因为policy iteration有两步：
     1. policy evaluation是在求当前策略下的Advantage
     2. policy improvement是用第一步得到的Advantage来更新得到新的策略
     - <font color=norange>**这两步合起来对应的数学表达就是**：？？？为什么数学上是这个形式，再理解policy iteration在做什么</font>
$$\underset{\bar \theta }{max}\ E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^tA^{\pi_{\theta}}(s_t,a_t) \right]$$

#### III.4.2 policy gradient的优化目标和上述policy iteration的优化目标近似

$$E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^tA^{\pi_{\theta}}(s_t,a_t) \right]
\approx E_{\tau\sim P_{\color{red}{\theta} }(\tau)}\left[\sum_t\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)\right]$$

- 证明：
  - 下式中的approximation成立是因为：当新旧策略差异不大的时候，可以认为$P_{\bar \theta}\approx P_{\theta}$

$$\begin{align}
E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^tA^{\pi_{\theta}}(s_t,a_t) \right]
&= \sum_tE_{s_t\sim P_{\bar \theta }(s_t)}[E_{a_t\sim \pi_{\bar \theta }(a_t|s_t)}[\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]]\\
&= \sum_tE_{s_t\sim P_{\bar \theta }(s_t)}[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]], importance\ sampling\\
&\approx \sum_tE_{s_t\sim P_{\color{red}{\theta} }(s_t)}[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]]\\
& = E_{\tau\sim P_{\color{red}{\theta} }(\tau)}\left[\sum_t\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)\right]
\end{align}$$

#### III.4.3 用policy gradient近似policy iteration的误差分析

- 当策略是deterministic时的bound分析
  - <img src='pics/bound_deterministic.png' width='70%'>

- 将策略一般化为任意分布时的bound分析
  - <img src='pics/bound_general.png' width='75%'>

- 根据下面的分析可知：只要$\epsilon$的值足够小，那么：
$$\sum_tE_{s_t\sim P_{\color{green}{\bar \theta}}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right] 
\approx \sum_tE_{s_t\sim P_{\color{red}{\theta}}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
$$
    - <img src='pics/policy_gradient_approx.png' width='75%'>

#### III.4.4 以误差bound为限制条件重写policy gradient的目标表达式

- 从上节的误差bound分析可知，当$\epsilon$很小时，下述有限制的优化目标一定可以improve$J(\bar\theta)-J(\theta)$。也就是说，policy gradient的目标在加上约束之后，就和policy iterate的目标完全一致了。
$$\begin{align}
& \underset{\bar \theta}{max} \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& s.t. |\pi_{\bar\theta }(a_t|s_t) - \pi_{\theta }(a_t|s_t)| \le \epsilon 
\end{align}$$

### III.5 带约束的policy gradient算法目标优化

#### III.5.1 Natural policy gradient将约束改为数学上更好处理的形式

- 前面有约束的PG优化目标的约束形式会让对应的优化问题不好处理，而natural PG算法提出的约束条件形式在数学上更好处理。
- 根据Pinsker's inequality, variational difference与KL divergence之间有下述关系：<font color=brown>[证明见附页，ref deepseek]</font>
$$|\pi_{\bar \theta}(a_t|s_t)-\pi_{\theta}(a_t|s_t)| \le \sqrt{0.5*D_{KL}(\pi_{\bar \theta}(a_t|s_t)||\pi_{\theta}(a_t|s_t))}$$
  - <font color=green>因此将前述约束条件转化为natural PG使用的KL divergence约束是同样有效的。</font>

- <font color=green>**当$\epsilon$很小时，下述有限制的优化目标同样能保证improve$J(\bar\theta)-J(\theta)$。**</font>
  - 使用KL divergence的好处是，其表达式方便求梯度，而使用绝对值的约束形式无法处处可导，因此这种方式在数学上更好处理。
  - 等价的优化目标：
$$\begin{align}
& \underset{\bar \theta}{max} \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& s.t. D_{KL}(\pi_{\bar\theta }(a_t|s_t) || \pi_{\theta }(a_t|s_t)) \le \epsilon 
\end{align}$$

#### III.5.2 泰勒一阶展开求解优化问题

$$\begin{align}
将优化目标简记为：\\
\underset{\bar \theta}{max} f(\bar \theta ) & = \underset{\bar \theta}{max} \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
f(\bar \theta )泰勒一阶展开：\\
f(\bar\theta ) & \approx f(\bar\theta _0) + \nabla_{\bar\theta }f(\bar\theta)|_{\bar\theta=\bar\theta _0}(\bar\theta -\bar\theta _0)\\
取\bar\theta _0=\theta,有：\\
f(\bar\theta ) & \approx f(\theta) + \nabla_{\theta }f(\theta)(\bar\theta -\theta)\\
因为f(\theta )与\bar\theta 无关：\\
\underset{\bar \theta}{max} f(\bar \theta ) & \approx \underset{\bar \theta}{max}\nabla_{\theta }f(\theta)(\bar\theta -\theta)
\end{align}$$

$$\begin{align}
\nabla_{\bar\theta }f(\bar\theta) 
& = \nabla_{\bar\theta }\sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& = \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\nabla_{\bar\theta }\pi_{\bar \theta}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& = \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \nabla_{\bar\theta }log\pi_{\bar \theta}(a_t|s_t)\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& {\color{blue}{正好是Importance\ sampling形式的策略梯度}}
\end{align}$$

- 取$\bar\theta _0=\theta$时,有：
$$
\begin{align}
f(\theta ) & =f(\bar \theta = \theta ) \\
&   = \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta=\theta}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& = \sum_tE_{s_t\sim P_{\theta}(s_t)}E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\\
& = E_{\tau\sim P_{\theta}(\tau)}\sum_t[\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\\
& = J(\theta )
\end{align}
$$

$$\begin{align}
\nabla_{\theta }f(\theta) & = \nabla_{\bar\theta }f(\bar\theta) |_{\bar\theta =\theta } \\
& = \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta=\theta}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \nabla_{\bar\theta }log\pi_{\bar \theta}(a_t|s_t)|_{\bar \theta=\theta}\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& = \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[ \nabla_{\theta }log\pi_{\theta}(a_t|s_t)\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& = \nabla_{\theta }J(\theta )
\end{align}$$

- 泰勒一阶展开后，原优化问题转化为：
$$\begin{align}
& \underset{\bar \theta}{max} f(\bar \theta ) \approx \underset{\bar \theta}{max}\nabla_{\theta }f(\theta)^T(\bar\theta -\theta)=\nabla_{\theta }J(\theta )^T(\bar\theta -\theta)\\
& s.t. D_{KL}(\pi_{\bar\theta }(a_t|s_t) || \pi_{\theta }(a_t|s_t)) \le \epsilon 
\end{align}$$

#### III.5.3 Natrual PG求解方法：用taylor一阶展开求解带约束的优化目标

- <font color=norange>**思路**：</font>
  1. gradient ascent方法是有约束的优化问题的solution，并且该优化问题中的约束是用L2 norm表达的: $\left\|\bar \theta-\theta\right\|^2 =  (\bar \theta-\theta)^TI(\bar \theta-\theta) \le \epsilon $
  2. KL divergence可以近似表达为Hessian matrix为Fisher-information matrix的矩阵形态：$D_{KL}(\pi_{\bar\theta}||\pi_{\theta}) \approx \frac{1}{2}(\bar \theta-\theta)^TF(\bar \theta-\theta)$
  3. 利用上面两种约束的相似性，可以将KL divergence处理到Gradient Ascend方法的step size中求解目标优化问题

1. Gradient Ascent的learning rate控制step size，本质上是给被迭代的参数加上了distance 限制。

$$\begin{align}
\underset{\bar \theta}{max} L(\bar \theta )&=\nabla_{\theta }J(\theta)^T(\bar \theta-\theta)\\
\nabla_{\bar \theta }L(\bar \theta ) &= \nabla_{\theta }J(\theta)\\
对应的GA迭代式: \\
\bar\theta & = \theta +\alpha \nabla_{\theta }J(\theta) \\
\end{align}$$

2. <font color=blue>**Gradient Ascent是有约束的优化问题的解**：</font>
    - 按照下面方式可以找到对应的有约束优化问题：<font color=brown>[详见附页 ref deepseek]</font>

$$\begin{align}
取\epsilon 与\alpha 的关系式：\\
\alpha &= \sqrt{\frac{\epsilon }{\left \| \nabla_\theta J(\theta ) \right \| ^2}} \nabla_\theta J(\theta )\\
迭代式是下面有约束目标问题的解：\\
\underset{\bar \theta}{max} L(\bar \theta )&=\nabla_{\theta }J(\theta)^T(\bar \theta-\theta)\\
s.t. \ \ \left\|\bar \theta-\theta\right\|^2& \le \epsilon 
\end{align}$$

3. 可以将KL divergence约束转化为类似L2 norm的形态
   - $D_{KL}(\pi_{\bar\theta}||\pi_{\theta}) \approx \frac{1}{2}(\bar \theta-\theta)^TF(\bar \theta-\theta)$  <font color=brown>[证明详见附页 ref deepseek]</font>
   - 其中，$F=E_{\pi_\theta}[\nabla_\theta log \pi_\theta (a|s)\nabla_\theta log \pi_\theta (a|s)^T]$，这个值可以抽样估计

4. 将原目标转化为Gradient ascent solution对应的目标，及其对应的GA solution为：
$$\begin{align}
\underset{\bar \theta}{max}& L(\bar \theta )=\nabla_{\theta }J(\theta)^T(\bar \theta-\theta)\\
s.t. \ \ &\frac{1}{2}(\bar \theta-\theta)^TF(\bar \theta-\theta) \le \epsilon \\
GA\ solution：\\
\theta '& = \theta +\alpha F^{-1}\nabla _\theta J(\theta )\\
\alpha  & = \sqrt{\frac{2\epsilon }{ \nabla_\theta J(\theta )^TF\nabla_\theta J(\theta ) }} 
\end{align}$$

#### III.5.4 TRPO提出了更高效的求解该优化问题的方法：共轭梯度

### III.6 PPO进一步简化优化目标，并用IS求解

- 梯度表达式直接对应的‘pseudo-loss’优化目标
$$\begin{align}
g = E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)A_t\right ] 
& \Rightarrow 
J(\theta ) = E_{\tau }\left[ \sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)A_t\right ]\\
\\
\hat g= \frac{1}{N} \sum _{i=1 }^N\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})A_t^{(i)}\right ] 
 & \Rightarrow 
\hat J(\theta ) = \frac{1}{N} \sum _{i=1 }^N\left[ \sum_{t=0}^{H-1}log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})A_t^{(i)}\right ]
\end{align}$$
  - 实践中不用，因为Advantage波动大导致策略迭代的波动大。